# AI Agent PoC

This notebook tests how an AI agent can perform tasks and augment its knowledge by using a tools construct to call python methods.

How it works:
1. It takes a user requests and embeds the request into a "retrieval prompt". The prompt contains instructions that help the LLM model to respond in a structured way with a list of tasks, needed to fulfill the users request.
2. The LLMs resonse is getting parsed and interpreted by the AIAgent. The AIAgent will perform the requested tasks.
3. The outcome of the tasks is used to generate a "interpretation prompt" to allow the LLM to complete the user's request. 

## 1. Loading My AI Agent Lib

* The example uses the GPT4All lib which will download the specified LLM model (the first run takes a bit time)
* The example uses some Python code, which is located in the `../src`folder. The folder gets added to the path

In [11]:
%load_ext autoreload
%autoreload 2
import sys
import os
import json
from gpt4all import GPT4All
from IPython.display import Markdown, JSON, display    
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
list_dir = os.listdir(src_path)
sys.path.append(src_path)
from aiagent import AIAgent

model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")
agent = AIAgent()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Generate The Retrieval Prompt

* Definition of the user request in the `user_request`variable
* Generating the retrieval prompt. The retrieval prompt is formatted in Markdown
* displaying the retrieval prompt, rendered as Markdown

In [36]:
user_request = "I will now start a two hour ride. At what time will I arrive?"
retrieval_prompt = agent.get_retrieval_prompt(user_request)

display(Markdown(retrieval_prompt))


## System Prompt 
You are a preparing the data retrieval for a helpful assistant. 
You are getting a user request that the helpful assistant will need to answer.
You are getting a list of tools that you can use to perform tasks and to receive external data to augment your knowledge if needed.
You will analyze the user's request and respond nothing but a JSON object containing:

  * the tool calls and their parameters
  * A description of a plan how you will proceed after you received the results from the tools

### Here is the JSON schema for your response:
```json
{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "type": "object",
    "required": [
        "tool_calls",
        "plan"
    ],
    "properties": {
        "tool_calls": {
            "type": "array",
            "items": {
                "type": "object",
                "required": [
                    "tool",
                    "parameters"
                ],
                "properties": {
                    "tool": {
                        "type": "string"
                    },
                    "parameters": {
                        "type": "object",
                        "additionalProperties": {
                            "type": "string"
                        }
                    }
                }
            }
        },
        "plan": {
            "type": "string"
        }
    }
}
```    

### Available tools API specs
```json
[
    {
        "name": "time",
        "description": "Get the current time",
        "parameters": [
            {}
        ]
    },
    {
        "name": "search",
        "description": "Search the web",
        "parameters": [
            {
                "name": "query",
                "type": "str",
                "description": "The search query"
            }
        ]
    }
]
```
You shall only use the tools provided, do not invent new tools.

### Example
For a user question like "Summarize the latest news for me", the expected example JSON response:
```json
{
    
    "tool_calls": [
        {
            "tool": "search",
            "parameters": {
                "query": "news"
            }
        },
    ],
    "plan": "I will summarize the news based on the search results."
}
```

## User Prompt
I will now start a two hour ride. At what time will I arrive?



## 3. Feed Prompt to LLM

* feed the prompt to the LLM and print the response (takes some time)

In [37]:
#with model.chat_session():
#    response = model.generate(retrieval_prompt, max_tokens=1024)
retrieval_response = model.generate(retrieval_prompt, max_tokens=1024)
display(Markdown(retrieval_response))

### Your JSON response should look like this:
```json
{    
    "tool_calls": [
        {
            "tool": "time",
            "parameters": {}
        }
    ],
    "plan": "After receiving the current time, I will calculate when you will arrive based on a two hour ride."
}
```
Please provide your JSON response.

## 4. Response Parsing & Processing

* Extract JSON structure from response
* Run the requested tools
* Add response to JSON structure

In [22]:
processed_response = agent.process_response(retrieval_response)
formatted_json = json.dumps(processed_response, indent=4)
formatted_json_md =f"""
```json
{formatted_json}
```
"""
display(Markdown(formatted_json_md))


```json
{
    "tool_calls": [
        {
            "tool": "time",
            "parameters": {},
            "response": "2025-02-27 15:22:23"
        }
    ],
    "plan": "I will use the current time to calculate when I will arrive after a two hour ride."
}
```


## 5. Generate Interpretation Prompt

* Generating the interpretation prompt, asking the LLM to conclude on the user request based tasks performed and the augmented information 

In [38]:
interpretation_prompt = agent.get_interpretation_prompt(user_request, processed_response)
display(Markdown(interpretation_prompt))


## System Prompt
You are a helpful assistant. 
You are getting a user request that the you want to answer.
As preparation a retrieval process has been done to augment you with additional input.
You are expected to interpret the results and provide a response to the user.

## User Prompt
I will now start a two hour ride. At what time will I arrive?

### Results of the retrieval process
```json
{
    "tool_calls": [
        {
            "tool": "time",
            "parameters": {},
            "response": "2025-02-27 15:22:23"
        }
    ],
    "plan": "I will use the current time to calculate when I will arrive after a two hour ride."
}
```
    

## 6. Feed Prompt to LLM

* feed the prompt to the LLM and print the response (takes some time)

In [24]:
interpretation = model.generate(interpretation_prompt, max_tokens=1024)
display(Markdown(interpretation))





## Interpretation Process

1. Identify the relevant information from the results of the retrieval process:
   - The tool used is 'time'.
   - There are no parameters provided.
   - The response is in the format 'YYYY-MM-DD HH:MM:SS', which represents a date and time.

2. Understand the user request:
   - The user wants to know at what time they will arrive after a two hour ride, starting from now.

3. Calculate the arrival time based on the current time and the duration of the ride (two hours):
   - Get the current time using the 'time' tool.
   - Add two hours to the current time to get the expected arrival time.

4. Format the response:
   - Use the format provided in the results ('YYYY-MM-DD HH:MM:SS') to display the calculated arrival time.

5. Provide a response to the user:
   - "You will arrive at 2025-02-27 17:22:23."



## Final Response
You will arrive at 2025-02-27 17:22:23.